# ADT feature file generation
## Qian Xiao

In [1]:
#Import libraries

import pandas as pd
import numpy as np
import xlrd
import openpyxl

In [5]:
work_order = 'KW9393'
local_path = '/Users/curtism/Documents/2021/singlecellcore/qian_scripts/input/{}'.format(work_order)

In [6]:
sheet_name = 'BRI-1412 Totalseq panel.xlsx'
adt_panel_path = '{local_path}/{sheet_name}'.format(local_path = local_path,
                                                    sheet_name = sheet_name)

In [7]:
adt_panel_path

'/Users/curtism/Documents/2021/singlecellcore/qian_scripts/input/KW9393/BRI-1412 Totalseq panel.xlsx'

In [8]:
df_panel_all = pd.read_excel(adt_panel_path, engine='openpyxl', sheet_name = 'TotalSeq-B')

In [9]:
df_panel_all.head()

,Catalog,Barcode,Description,Unique Description,Clone,Reactivity,Sequence,Is it in your panel?
0,100573,1,CD4,0001CD4,RM4-5,Mouse,AACAAGACCCTTGAG,NaN
1,100783,2,CD8a,0002CD8a,53-6.7,Mouse,TACCCGTAATAGCGT,NaN
2,119741,3,CD366,0003CD366,RMT3-23,Mouse,ATTGGCACTCAGATG,NaN
3,109125,4,CD279,0004CD279,RMP1-30,Mouse,GAAAGTCAAAGCACT,NaN
4,305241,5,CD80,0005CD80,2D10,"Human, <B>Cross-Reactivity:</B> Rhesus",ACGAATCAATCTGTG,NaN


In [10]:
df_panel = df_panel_all

In [11]:
###For the type of panel spread sheet in which the useres mark Y when they want to include the antibody
df_panel = df_panel_all[df_panel_all['Is it in your panel?'] == 'Y']

In [29]:
name = df_panel.Description.tolist()
sequence = df_panel.Barcode.tolist()  # May be named sequence if Y spread sheet
# sequence = df_panel.Sequence.tolist()
clone = df_panel.Clone.tolist()

In [9]:
###The panel could be named differently, so consider use these below when a differently named sheet is provided


#name = df_panel['Gene Name'].tolist()
#sequence = df_panel.Barcode.tolist()
#clone = df_panel.Clone.tolist()

In [16]:
#name = df_panel.Description.tolist()
#sequence = df_panel.Sequence.tolist()
#clone = df_panel.Clone.tolist()

In [13]:
df_panel.head()

,Catalog,Barcode,Description,Unique Description,Clone,Reactivity,Sequence,Is it in your panel?
37,313247,61,CD117,0061CD117,104D2,"Human, <B>Cross-Reactivity:</B> Cynomolgus, Ca...",AGACTAATAGCTGAC,Y
62,400185,90,"IgG1, k Isotype Ctrl","0090IgG1, k Isotype Ctrl",MOPC-21,NaN,GCCGGACGACATTAA,Y
63,400291,91,"IgG2a, k isotype Ctrl","0091IgG2a, k isotype Ctrl",MOPC-173,NaN,CTCCTACCTAAACTG,Y
64,400379,92,"IgG2b, k Isotype Ctrl","0092IgG2b, k Isotype Ctrl",MPC-11,NaN,ATATGTATCACGCGA,Y
102,350235,145,CD103,0145CD103,Ber-ACT8,"Human, Cynomolgus",GACCTCATTGTGAAT,Y


In [14]:
#This is for the case like the sample cite-seq panel, when there are additional description like '0209 anti-mouse' before the actual protein name,
#So would need to get rid of those first
inter_name = []
for i in range(len(name)):
    short = name[i].split(' ')[2:len(name[i].split(' '))]
    short_str = ''.join(short)
    inter_name.append(short_str)

In [ ]:
##In case we don't need to generate the inter_name, just use inter_name = name to directly begin the special character removal process

In [15]:
inter_name[0:4]

['', 'IsotypeCtrl', 'isotypeCtrl', 'IsotypeCtrl']

In [18]:
inter_name = name
inter_name[0:4]

['CD117',
 'IgG1, k Isotype Ctrl',
 'IgG2a, k isotype Ctrl',
 'IgG2b, k Isotype Ctrl']

In [19]:
#Then we need to remove the special characters that cellranger can't recognize
new_name = []
for i in inter_name:
    #print(i)
    i = i.replace("anti-human/mouse/rat", "")
    i = i.replace("anti-human/mouse", "")
    i = i.replace("anti-mouse/rat", "")
    i = i.replace("anti-mouse/human", "")
    #i = i.replace("Mouse ", "")
    i = i.replace("anti-mouse", "")
    i = i.replace("anti-human", "")
    i = i.replace(" ", "")
    i = i.replace("'", "")
    i = i.replace("/", "_or_")
    i = i.replace("(", "_or_")
    i = i.replace(")", "")
    i = i.replace(",", "_")
    i = i.replace("α", "a")
    i = i.replace("β", "b")
    i = i.replace("κ", "k")
    i = i.replace("λ", "lambda")
    i = i.replace("δ", "delta")
    i = i.replace("γ", "gamma")
    i = i.replace("ε", "e")  
    i = i + "_prot"
    new_name.append(i)
    #print(i)

In [20]:
#Here just showing 2 of the names, but always take a look at the full list to make sure the names look good.
new_name[0:4]

['CD117_prot',
 'IgG1_kIsotypeCtrl_prot',
 'IgG2a_kisotypeCtrl_prot',
 'IgG2b_kIsotypeCtrl_prot']

In [21]:
new_name

['CD117_prot',
 'IgG1_kIsotypeCtrl_prot',
 'IgG2a_kisotypeCtrl_prot',
 'IgG2b_kIsotypeCtrl_prot',
 'CD103_prot',
 'Integrinb7_prot',
 'IgG2a_kIsotypeCtrl_prot',
 'FceRIa_prot',
 'CD184_prot',
 'CD71_prot',
 'CD38_prot',
 'CD49d_prot',
 'CD133_prot',
 'Biotin_prot']

In [22]:
#Check if there are any duplicated names in there, if so, add clone name after it to avoid repeated description
dup_name = []
if len(new_name) == len(set(new_name)):
    new_name = new_name
else:
    import collections
    dup_name = [item for item, count in collections.Counter(new_name).items() if count > 1]
    for i in range(len(new_name)):
        if new_name[i] in dup_name:
            new_name[i] = new_name[i] + '_' + clone[i]
        

In [23]:
dup_name

[]

In [24]:
new_name

['CD117_prot',
 'IgG1_kIsotypeCtrl_prot',
 'IgG2a_kisotypeCtrl_prot',
 'IgG2b_kIsotypeCtrl_prot',
 'CD103_prot',
 'Integrinb7_prot',
 'IgG2a_kIsotypeCtrl_prot',
 'FceRIa_prot',
 'CD184_prot',
 'CD71_prot',
 'CD38_prot',
 'CD49d_prot',
 'CD133_prot',
 'Biotin_prot']

In [40]:
##Run this cell if 
#Totalseq-A
dict_features = {
    'id' : new_name,
    'name' : new_name,
    'read' : 'R2',
    'pattern' : '5P(BC)',
    'sequence' : sequence,
    'feature_type' : 'Antibody Capture'
}

In [30]:
##Run this cell if
#Totalseq-B or C
dict_features = {
    'id' : new_name,
    'name' : new_name,
    'read' : 'R2',
    'pattern' : '5PNNNNNNNNNN(BC)',
    'sequence' : sequence,
    'feature_type' : 'Antibody Capture'
}

In [31]:
df_features = pd.DataFrame(data = dict_features)

In [32]:
df_features

,id,name,read,pattern,sequence,feature_type
0,CD117_prot,CD117_prot,R2,5PNNNNNNNNNN(BC),AGACTAATAGCTGAC,Antibody Capture
1,IgG1_kIsotypeCtrl_prot,IgG1_kIsotypeCtrl_prot,R2,5PNNNNNNNNNN(BC),GCCGGACGACATTAA,Antibody Capture
2,IgG2a_kisotypeCtrl_prot,IgG2a_kisotypeCtrl_prot,R2,5PNNNNNNNNNN(BC),CTCCTACCTAAACTG,Antibody Capture
3,IgG2b_kIsotypeCtrl_prot,IgG2b_kIsotypeCtrl_prot,R2,5PNNNNNNNNNN(BC),ATATGTATCACGCGA,Antibody Capture
4,CD103_prot,CD103_prot,R2,5PNNNNNNNNNN(BC),GACCTCATTGTGAAT,Antibody Capture
5,Integrinb7_prot,Integrinb7_prot,R2,5PNNNNNNNNNN(BC),TCCTTGGATGTACCG,Antibody Capture
6,IgG2a_kIsotypeCtrl_prot,IgG2a_kIsotypeCtrl_prot,R2,5PNNNNNNNNNN(BC),AAGTCAGGTTCGTTT,Antibody Capture
7,FceRIa_prot,FceRIa_prot,R2,5PNNNNNNNNNN(BC),CTCGTTTCCGTATCG,Antibody Capture
8,CD184_prot,CD184_prot,R2,5PNNNNNNNNNN(BC),TCAGGTCCTTTCAAC,Antibody Capture
9,CD71_prot,CD71_prot,R2,5PNNNNNNNNNN(BC),CCGTGTTCCTCATTA,Antibody Capture


In [33]:
#Save it to .csv under the local path you specified, you may name it differently, but remember to change it accordingly in your count feature parameters
df_features.to_csv('{}/features_ADT_BRI-1413.csv'.format(local_path), index = False)